In [1]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import SVG

import cv2
from keras.applications import *
from keras.applications.inception_v3 import preprocess_input
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.regularizers import *
from keras.utils.vis_utils import model_to_dot
from tqdm import tqdm


Using TensorFlow backend.


In [16]:
# Loading Datasets

df = pd.read_csv('../dog_breed_datasets/labels.csv')
df.head()

n = len(df)
breed = set(df['breed'])
n_class = len(breed)
class_to_num = dict(zip(breed, range(n_class)))
num_to_class = dict(zip(range(n_class), breed))

width = 299
X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, n_class), dtype=np.uint8)
for i in tqdm(range(n)):
    X[i] = cv2.resize(
        cv2.imread('../dog_breed_datasets/train/%s.jpg' % df['id'][i]),
        (width, width))
    y[i][class_to_num[df['breed'][i]]] = 1


100%|██████████| 10222/10222 [00:29<00:00, 349.40it/s]


In [13]:
# Data visualization
#
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
#
# plt.figure(figsize=(12, 6))
# for i in range(8):
#     random_index = random.randint(0, n-1)
#     plt.subplot(2, 4, i+1)
#     plt.imshow(X[random_index][:,:,::-1])
#     plt.title(num_to_class[y[random_index].argmax()])
#


In [17]:
# Computing feature

def get_features(MODEL, data=X):
    cnn_model = MODEL(
        include_top=False, input_shape=(width, width, 3), weights='imagenet')

    inputs = Input((width, width, 3))
    x = inputs
    x = Lambda(preprocess_input, name='preprocessing')(x)
    x = cnn_model(x)
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs, x)

    features = cnn_model.predict(data, batch_size=64, verbose=1)
    return features


print('Start computing inception_features: ')
inception_features = get_features(InceptionV3, X)
print('Start computing xception_features: ')
xception_features = get_features(Xception, X)
features = np.concatenate([inception_features, xception_features], axis=-1)


Start computing inception_features: 
10222/10222 [==============================] - 96s 9ms/step
Start computing xception_features: 
10222/10222 [==============================] - 159s 16ms/step


In [29]:
features.shape

(10222, 4096)

In [30]:
# Training models
inputs = Input(features.shape[1:])
x = inputs
x = Dropout(0.5)(x)
# x = Dense(4096, activation='relu', name='fc1')(x)
# x = Dense(4096, activation='relu', name='fc2')(x)
# x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dense(n_class, activation='softmax', name='predictions')(x)
model = Model(inputs, x)
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
h = model.fit(features, y, batch_size=128, epochs=10, validation_split=0.1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/10
9199/9199 [==============================] - 2s 238us/step - loss: 1.8820 - acc: 0.5969 - val_loss: 0.7006 - val_acc: 0.8240
Epoch 2/10
9199/9199 [==============================] - 0s 29us/step - loss: 0.5803 - acc: 0.8360 - val_loss: 0.5592 - val_acc: 0.8270
Epoch 3/10
9199/9199 [==============================] - 0s 28us/step - loss: 0.4417 - acc: 0.8692 - val_loss: 0.4985 - val_acc: 0.8465
Epoch 4/10
9199/9199 [==============================] - 0s 28us/step - loss: 0.3598 - acc: 0.8954 - val_loss: 0.4936 - val_acc: 0.8426
Epoch 5/10
9199/9199 [==============================] - 0s 28us/step - loss: 0.3040 - acc: 0.9130 - val_loss: 0.4707 - val_acc: 0.8514
Epoch 6/10
9199/9199 [==============================] - 0s 28us/step - loss: 0.2600 - acc: 0.9272 - val_loss: 0.4662 - val_acc: 0.8583
Epoch 7/10
9199/9199 [==============================] - 0s 28us/step - loss: 0.2277 - acc: 0.9354 - val_loss: 0.4692 - val_acc: 0.8524
Epoch 

In [32]:
# Loading test Datasets
df2 = pd.read_csv('../dog_breed_datasets/sample_submission.csv')

n_test = len(df2)
X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
for i in tqdm(range(n_test)):
    X_test[i] = cv2.resize(
        cv2.imread('../dog_breed_datasets/test/%s.jpg' % df2['id'][i]),
        (width, width))

# Compute test dataset features
print('Start computing inception_features: ')
inception_features = get_features(InceptionV3, X_test)
print('Start computing xception_features: ')
xception_features = get_features(Xception, X_test)

features_test = np.concatenate(
    [inception_features, xception_features], axis=-1)

# predict
y_pred = model.predict(features_test, batch_size=128)

for b in breed:
    df2[b] = y_pred[:, class_to_num[b]]

df2.to_csv('pred.csv', index=None)

100%|██████████| 10357/10357 [00:26<00:00, 390.50it/s]


Start computing inception_features: 
10357/10357 [==============================] - 100s 10ms/step
Start computing xception_features: 
10357/10357 [==============================] - 163s 16ms/step
